<a href="https://colab.research.google.com/github/filipes1lva/Projeto_Oean/blob/main/RKU_ANN_STCond.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
pip install -q -U keras-tuner

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import Dense

from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from sklearn.preprocessing import MinMaxScaler

**Carregando base de dados**

In [ ]:
data = pd.read_csv('/Exp2.csv')
data_prep = data.copy()

data_prep.head()

,Run_ID,Experiment,Replica,Tool_ID,Group,Subgroup,Position,Condition,TCond,Machined_length,...,Ra,Rz,Rsk,Rku,RSm,Rt,Fx,Fy,Fz,F
0,2_043_L1_4_a,2,1,43,1,na,a,4,0.0,12,...,1.050,4.504,0.959,2.728,132.5,4.864,80.23,61.8,28.33,105.160267
1,2_043_L1_4_a,2,1,43,1,na,a,4,0.0,12,...,1.088,4.791,0.904,2.637,129.7,5.776,80.23,61.8,28.33,105.160267
2,2_043_L1_4_a,2,1,43,1,na,a,4,0.0,12,...,1.056,4.451,0.984,2.741,130.9,4.696,80.23,61.8,28.33,105.160267
3,2_043_L1_4_a,2,1,43,1,na,a,4,0.0,12,...,1.059,4.518,0.990,2.758,127.3,4.802,80.23,61.8,28.33,105.160267
4,2_043_L1_4_a,2,1,43,1,na,a,4,0.0,12,...,1.062,4.578,1.025,2.838,128.3,4.814,80.23,61.8,28.33,105.160267


**Limpeza da base de dados**

In [ ]:
data_prep.drop(['Run_ID', 'Experiment', 'Replica', 'Tool_ID', 'Group', 'Subgroup',
           'Position', 'Condition',
           'Machined_length', 'Init_diameter', 'Final_diameter', 'CTime', 'R_measurement'],
          axis=1, inplace=True)

data_prep.head(3)

,TCond,ap,vc,f,Ra,Rz,Rsk,Rku,RSm,Rt,Fx,Fy,Fz,F
0,0.0,0.25,350,0.13,1.050,4.504,0.959,2.728,132.5,4.864,80.23,61.8,28.33,105.160267
1,0.0,0.25,350,0.13,1.088,4.791,0.904,2.637,129.7,5.776,80.23,61.8,28.33,105.160267
2,0.0,0.25,350,0.13,1.056,4.451,0.984,2.741,130.9,4.696,80.23,61.8,28.33,105.160267


**Separação dos dados de treino e teste**

In [ ]:
sep = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=21)

for train_index, test_index in sep.split(data_prep, data_prep["f"]):
    stratified_train_set = data_prep.loc[train_index]
    stratified_test_set = data_prep.loc[test_index]
    
df_train = pd.DataFrame(stratified_train_set)
df_test = pd.DataFrame(stratified_test_set)

df_train.head(3)

,TCond,ap,vc,f,Ra,Rz,Rsk,Rku,RSm,Rt,Fx,Fy,Fz,F
164,0.1,0.50,350,0.13,0.820,3.544,0.585,2.328,132.7,3.939,185.20,117.60,91.08,237.538137
287,0.3,0.50,350,0.09,0.473,2.958,1.001,3.859,78.5,3.196,195.10,221.20,174.90,342.904447
41,0.0,0.25,350,0.07,0.155,0.816,0.209,2.407,35.7,0.975,55.29,44.64,24.17,75.059327


**Definição variáveis de entrada**

In [ ]:
#4.1 - Variáveis de Entrada
X_train_and_valid = df_train.drop(["Ra", "Rz", "Rsk", "Rku", "RSm", "Rt"], axis=1)  
X_test = df_test.drop(["Ra", "Rz", "Rsk", "Rku", "RSm", "Rt"], axis=1)

#4.2 - Variável de Saída - Kurtose
y_train = df_train[["Rku"]]  
y_test = df_test[["Rku"]] 


y_test.shape
X_test.head()

,TCond,ap,vc,f,Fx,Fy,Fz,F
166,0.1,0.50,350,0.13,185.2,117.60,91.08,237.538137
236,0.3,0.25,350,0.13,127.8,181.10,88.08,238.512340
65,0.0,0.50,350,0.09,125.1,65.95,67.73,156.801675
72,0.0,0.50,350,0.07,104.3,55.74,60.61,132.887207
188,0.1,0.50,350,0.11,161.3,100.50,84.26,207.888642


**Separação dos dados de treino e validação**

In [ ]:
#5 - Separação dos dados de validação e treino

X_train, X_valid,\
y_train, y_valid = train_test_split(X_train_and_valid,y_train,
                                  test_size=0.20, random_state=21)

#5.1 - Normalização dos dados

scaler =  MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_valid=scaler.fit_transform(X_valid)
X_test=scaler.fit_transform(X_test)


#Obs.: Tamanho do conjuntos de dados das variáveis de entrada

X_train.shape

(184, 8)

In [ ]:
def model_create(hp):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape=(8)))

  hp_activation = hp.Choice('activation', values=['relu', 'sigmoid'])
  hp_layer_1 = hp.Int('layer_1', min_value=1, max_value=100, step=10)
  hp_layer_2 = hp.Int('layer_2', min_value=1, max_value=100, step=10)
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4 ])


  model.add(tf.keras.layers.Dense(units=hp_layers_1, activation=hp_activation))
  model.add(tf.keras.layers.Dense(units=hp_layers_2, activation=hp_activation))
  model.add(tf.keras.layers.Dense(10, activation='softmax'))

  model.compile(optmizer=tf.keras.optmizers.Adam(learning_rate=hp_learning_rate,
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics=['accuracy']))
  
  return model

In [ ]:
import keras_tuner as kt

tuner = kt.Hyperband(model_create, 
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='dir',
                     project_name='x')



TypeError: ignored